In [1]:
%load_ext autoreload
%autoreload 2
%env ANYWIDGET_HMR=1

env: ANYWIDGET_HMR=1


In [2]:
# Don't run this cell when debugging from VS Code, only when running the notebook with Jupyter lab.
import debugpy
debugpy.listen(5678)
print("Waiting for debugger attach...")
debugpy.wait_for_client()
print("Debugger attached.")

Waiting for debugger attach...
Debugger attached.


In [ ]:
import urllib.request
import os.path

# Setup data
data_url = "https://www.dropbox.com/scl/fi/ih2laraxptbaeqfgqfax9/pbmc3k_20210420.h5ad?rlkey=fdtrijj9rh971uwvaa5x4qaq9&st=o0g4sf07&dl=1"
data_file = "data/pbmc3k_20210420.h5ad"

if not os.path.exists(data_file):
    os.mkdir("data")
    print("Downloading PBMC 3K demo...")
    urllib.request.urlretrieve(data_url, data_file)
    print("Download complete.")

In [ ]:
import scanpy as sc

adata = sc.read(data_file)
adata

In [ ]:
from scsketch import ScSketch

sketch = ScSketch(
    adata=adata,
    # categorical_columns=categorical_cols,
    # color_by_default="seurat_clusters",
    height=720,
    background_color="#111111",
)

sketch.show()